#### Installation

In [1]:
!pip install git+https://github.com/openai/CLIP.git
!pip install kaolin==0.17.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
!pip install open-clip-torch

!pip install open3d

import os

if not os.path.exists("AML_group"):
    !git clone https://github.com/AlesCarl/AML_group.git
else:
    print("Repository già clonato.")
%cd AML_group

from google.colab import drive
drive.mount('/content/drive')

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-e2wiikcy
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-e2wiikcy
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=9d2caf7084d9ffae148682c72666b102625cb250b4ad83ef7e0ebdbcd8675949
  Stored in directory: /tmp/pip-ephem-wheel-cache-2eo7umff/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip
Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━

#### Imports

In [2]:
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
import open_clip
import open3d as o3d
import numpy as np
import h5py
import json
import pickle as pkl


from itertools import permutations, product
from Normalization import MeshNormalizer
from utils import device, color_mesh
from mesh import Mesh
from render import Renderer

from pathlib import Path
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.transforms import functional as F
from PIL import Image


seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

Warp 1.5.1 initialized:
   CUDA Toolkit 12.6, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.5.1


# Neural Highlighter Class

In [3]:
class NeuralHighlighter(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(NeuralHighlighter, self).__init__()

        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm([hidden_dim]))

        layers.append(nn.Linear(hidden_dim, output_dim))
        layers.append(nn.Softmax(dim=1))

        self.mlp = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.mlp:
            x = layer(x)
        return x

## Utility Functions

In [4]:
def get_clip_model(clip_model):
    device = 'cuda'
    model, preprocess = clip.load(clip_model, device=device) # jit = True for better perfomance
    return model

def save_final_results(log_dir, name, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)

        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=5, show=False, center_azim=0, center_elev=0, std=1, return_views=True, lighting=True, background=background)

        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        mesh.export(os.path.join(log_dir, f"{name}.ply"), extension="ply", color=final_color)
        save_renders(log_dir, 0, rendered_images, name='final_render.jpg')

def clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, clip_model):
    if n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if encoded_text.shape[0] > 1:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
        else:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)
    elif n_augs > 0:
        loss = 0.0
        for _ in range(n_augs):
            augmented_image = augment_transform(rendered_images)
            #print(augmented_image.shape)
            encoded_renders = clip_model.encode_image(augmented_image)
            if encoded_text.shape[0] > 1:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
        loss= loss / n_augs
    return loss

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))

# Optimizer Loop

In [5]:
def optimize(obj_path, learning_rate, n_layers, n_views, n_augs, prompt, augment_transform=None):
    render_res = 224
    n_iter = 1000
    res = 224
    output_dir = './output/'
    clip_model = 'ViT-L/14'

    input_dim = 3
    hidden_dim = 256
    output_dim = 2

    Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

    objbase, extension = os.path.splitext(os.path.basename(obj_path))

    #render = Renderer(dim=(render_res, render_res), background_image= None )
    render = Renderer(dim=(render_res, render_res), background_image='./data/desk1.jpg')

    mesh = Mesh(obj_path)
    MeshNormalizer(mesh)()

    # Initialize variables
    background = torch.tensor((1., 1., 1.)).to(device)

    # CLIP and augmentation transform
    clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
    ])
    if augment_transform is None:
        augment_transform = transforms.Compose([
            transforms.RandomResizedCrop(res, scale=(1, 1)),
            transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
            clip_normalizer
        ])

    # MLP and optimizer Settings
    mlp = NeuralHighlighter(input_dim, hidden_dim, output_dim, n_layers).to(device)
    optim = torch.optim.Adam(mlp.parameters(), learning_rate)

    # List of possible colors
    rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
    color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
    full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
    colors = torch.tensor(full_colors).to(device)

    # Encode prompt with CLIP
    model = get_clip_model(clip_model)

    with torch.no_grad():
        prompt_token = clip.tokenize([prompt]).to(device)
        encoded_text = model.encode_text(prompt_token)
        encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

    vertices = copy.deepcopy(mesh.vertices)

    losses = []

    # Optimization loop
    for i in tqdm(range(n_iter)):
        optim.zero_grad()

        # predict highlight probabilities
        pred_class = mlp(vertices)

        # color and render mesh
        sampled_mesh = mesh
        color_mesh(pred_class, sampled_mesh, colors)
        rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=n_views, show=False, center_azim=0, center_elev=0, std=1, return_views=True, lighting=True, background=background) #,background=background)

        # Calculate CLIP Loss
        loss = clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, model)
        loss.backward(retain_graph=True)

        optim.step()

        # update variables + record loss
        with torch.no_grad():
            losses.append(loss.item())

        # report results
        if i % 100 == 0:
            print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
            save_renders(output_dir, i, rendered_images)
            with open(os.path.join(output_dir, "training_info.txt"), "a") as f:
                f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")


    # save results
    save_final_results(output_dir, 'final_mesh', mesh, mlp, vertices, colors, render, background) ##

    return mlp

# First Part

In [ ]:
obj_path = 'data/horse.obj'

learning_rate = 0.0001
n_layers = 5
n_views = 4
n_augs = 4

prompt= "A 3D rendering of a Horse with highlighted Shoes."

model_instance = optimize(obj_path, learning_rate, n_layers, n_views, n_augs, prompt)

Background image path:  ./data/bg1.jpg


 21%|████████                               | 185M/890M [00:05<00:20, 36.1MiB/s]

KeyboardInterrupt



# Second Part

## Mesh 2 Point Cloud

In [12]:
# Percorso del file OBJ
obj_path = 'data/horse.obj'

# Funzione per caricare la mesh come TriangleMesh
def load_obj_as_triangle_mesh(obj_path):
    # Carica il file .obj
    mesh = o3d.io.read_triangle_mesh(obj_path)
    if mesh.is_empty():
        raise ValueError(f"La mesh nel file {obj_path} non è stata caricata correttamente.")
    return mesh

# Carica la mesh
mesh = load_obj_as_triangle_mesh(obj_path)

# Converte la mesh in una point cloud campionando punti uniformemente
pcd = mesh.sample_points_uniformly(2048)

# Imposta il colore nero per tutti i punti
pcd.colors = o3d.utility.Vector3dVector(np.zeros((len(pcd.points), 3)))  # Colore nero: [0, 0, 0]

# Esporta la point cloud in formato PLY
# o3d.io.write_point_cloud("candle_PC.ply", pcd)

# Point cloud to mesh

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)

o3d.io.write_triangle_mesh("APPROXIMATE.obj", mesh)

[Open3D DEBUG] Input Points / Samples: 2048 / 2046
[Open3D DEBUG] #   Got kernel density: 0.01088404655456543 (s), 3936.60546875 (MB) / 3936.60546875 (MB) / 4028 (MB)
[Open3D DEBUG] #     Got normal field: 0.008194923400878906 (s), 3937.4296875 (MB) / 3937.4296875 (MB) / 4028 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 5.753411e-04 / 1.178298e+00
[Open3D DEBUG] #       Finalized tree: 0.02439403533935547 (s), 3937.6875 (MB) / 3937.6875 (MB) / 4028 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.15444207191467285 (s), 3937.81640625 (MB) / 3937.81640625 (MB) / 4028 (MB)
[Open3D DEBUG] #Set point constraints: 0.007436990737915039 (s), 3937.81640625 (MB) / 3937.81640625 (MB) / 4028 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 153931 / 88248 / 87673
[Open3D DEBUG] Memory Usage: 3937.816 MB
[Open3D DEBUG] # Linear system solved: 0.13541483879089355 (s), 3937.8984375 (MB) / 3937.8984375 (MB) / 4028 (MB)
[Open3D DEBUG] Got average: 0.005606889724731445 (s), 3937.8984375 (M

True

## Point Cloud 2 Mesh

In [14]:
def pointToMesh(data, output_path):
    single_data = data[0].cpu().numpy()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(single_data)

    # Stima delle normali
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.06,  # Raggio per la stima delle normali,
        max_nn=80     # Numero massimo di vicini per stimare la normale
    ))

    # Orienta le normali in modo consistente
    pcd.orient_normals_consistent_tangent_plane(k=100)

    # Ricostruzione con
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
        pcd, 0.04
    )

    # Salva e visualizza la mesh
    o3d.io.write_triangle_mesh(output_path, mesh)

# Third Part

## AffordanceNet Class to handle the dataset

In [8]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc, centroid, m


class AffordNetDataset(Dataset):
    def __init__(self, data_dir, split):
        super().__init__()
        self.data_dir = data_dir
        self.split = split

        self.load_data()
        self.affordance = self.all_data[0]["affordance"]
        return

    def load_data(self):
     self.all_data = []
     with open(os.path.join(self.data_dir, 'full_shape_%s_data.pkl' % self.split), 'rb') as f:
        temp_data = pkl.load(f)
     for index, info in enumerate(temp_data):
        if info["semantic class"] == "Scissors":  # Filtra solo gli oggetti "BOWL"
            temp_info = {}
            temp_info["shape_id"] = info["shape_id"]
            temp_info["semantic class"] = info["semantic class"]
            temp_info["affordance"] = info["affordance"]
            temp_info["data_info"] = info["full_shape"] # vertici
            self.all_data.append(temp_info)


    def __getitem__(self, index):
        data_dict = self.all_data[index]
        modelid = data_dict["shape_id"]
        modelcat = data_dict["semantic class"]

        data_info = data_dict["data_info"]
        model_data = data_info["coordinate"].astype(np.float32)
        labels = data_info["label"]
        for aff in self.affordance:
            temp = labels[aff].astype(np.float32).reshape(-1, 1)
            model_data = np.concatenate((model_data, temp), axis=1)

        datas = model_data[:, :3]
        targets = model_data[:, 3:]
        datas, _, _ = pc_normalize(datas)

        return datas, datas, targets, modelid, modelcat

    def __len__(self):
        return len(self.all_data)




    def get_random_sample(self):
        index = random.randint(0, len(self.all_data) - 1)
        return self.__getitem__(index)



def build_dataset(data_dir, test=False):
    test_set = AffordNetDataset(data_dir, 'train')
    val_set = AffordNetDataset(data_dir, 'val')

    # Seleziona i primi 5 campioni per il VAL set
    ### val_indices = list(range(min(6, len(val_set)))) # uno in più
    val_indices = random.sample(range(len(val_set)), min(6, len(val_set))) # casuali

    val_set = Subset(val_set, val_indices)

    # Seleziona 10 campioni per il TEST set, a partire dal 7° (indice 6)
    test_start_index = 6
    test_indices = list(range(test_start_index, test_start_index + min(10, len(test_set) - test_start_index)))
    test_set = Subset(test_set, test_indices)

    # Ritorna i dataset in un dizionario
    dataset_dict = dict(val_set=val_set, test_set=test_set)
    return dataset_dict

def build_loader(dataset_dict):
    val_set = dataset_dict["val_set"]
    test_set = dataset_dict["test_set"]

    batch_size_factor = 1

    test_loader = DataLoader(test_set, batch_size=1, shuffle=True, drop_last=False, num_workers=8)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=8, drop_last=False)
    #random_sample = val_set.dataset.get_random_sample() # new


    loader_dict = dict(val_loader=val_loader, test_loader=test_loader) # new
    return loader_dict

## Evaluate the mIOU

In [9]:
def calculate_miou(predictions, ground_truths, threshold):
    ground_truths = (ground_truths >= threshold).astype(int)  # Binarizza ground truth

    batch_size, num_points = ground_truths.shape
    iou_per_class = np.zeros((batch_size, 1))

    for b in range(batch_size):
        pred = predictions[b, :]
        gt = ground_truths[b, :]

        # Calcola intersezione e unione
        intersection = np.sum(pred * gt)
        union = np.sum(pred + gt) - intersection

        if union == 0:  # Evita divisione per zero
            iou_per_class[b, 0] = np.nan  # Non valido se non ci sono punti
        else:
            iou_per_class[b, 0] = intersection / union

    # Media su batch e classi
    mean_iou = np.nanmean(iou_per_class)
    return mean_iou

## Load the dataset

In [10]:
# dataset_dir = '/content/drive/My Drive/full-shape'
dataset_dir = '/content/drive/My Drive/ColabNotebooks'

dataset = build_dataset(dataset_dir)

loader = build_loader(dataset)



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Run

## Training + Validation


In [ ]:
model_instance = None


learning_rate = 0.0001
n_layers = 4
n_views = 4
n_augs = 3



#prompt= "A 3D rendering of a pair of scissors, emphasizing the handles, which are shaped for a one-handed grip" # training di una sola
prompt= "A 3D rendering of a pair of scissors, emphasizing the ergonomic handles, which are shaped for a firm and comfortable one-handed grip"



# Ottieni il primo batch dal val_loader
first_batch = next(iter(loader["val_loader"]))

data, data1, targets, modelid, modelcat = first_batch
vertex = data
obj_path = 'alpha.obj'
pointToMesh(data, obj_path)

model_instance = optimize(obj_path, learning_rate, n_layers, n_views, n_augs, prompt) # 1 solo training




Background image path:  None


  0%|          | 1/1000 [00:00<11:09,  1.49it/s]

Last 100 CLIP score: -0.2412109375


 10%|█         | 101/1000 [01:04<09:44,  1.54it/s]

Last 100 CLIP score: -0.257255859375


...

 80%|████████  | 801/1000 [08:41<02:10,  1.53it/s]

Last 100 CLIP score: -0.250093994140625


 90%|█████████ | 901/1000 [09:46<01:04,  1.52it/s]

Last 100 CLIP score: -0.24994384765625


100%|██████████| 1000/1000 [10:52<00:00,  1.53it/s]


In [ ]:
# nuovo eval


model_instance.eval()  # imposta il modello in modalità valutazione

all_mious = []

with torch.no_grad():  # disabilita il calcolo dei gradienti per l'inferenza

    for batch_idx, batch in enumerate(loader["val_loader"]):

        if batch_idx == 1: #questo perchè il primo valore lo uso per il train ( VA BENE ???????--- TODO ############)
         continue

        # Il tuo batch può restituire più elementi; qui estraiamo ciò che serve
        data, data1, targets, modelid, modelcat = batch

        # Sposta su GPU (o sul device corretto) e converti in float se necessario
        data = data.float().cuda()
        targets = targets.float().cuda()

        # Inferenza con il modello
        afford_pred = model_instance(data)   # [batch_size, num_points, 2] ad es.

        # Se la tua uscita è di shape [B, N, 2], prendi l'argmax per ricavare la classe
        afford_pred = torch.argmax(afford_pred, dim=-1)  # Shape: [1, 2048]

        afford_pred = afford_pred ^ 1


        new_targets = targets[:, :, 0]

        # Convertilo su CPU per calcolo mIoU (se la tua funzione lo richiede)
        afford_pred_cpu = afford_pred.detach().cpu().numpy()
        ground_truth = new_targets.detach().cpu().numpy()  # [B, N]

        # Calcolo del mIOU (supponendo che la tua funzione handle batch e shape correttamente)
        miou = calculate_miou(afford_pred_cpu, ground_truth, threshold=0.05)
        all_mious.append(miou)

# Al termine, puoi calcolare la media delle metriche
mean_miou = np.mean(all_mious)
print(f" IOU (val set): {all_mious}")
print(f"Mean IOU (val set): {mean_miou}")

 IOU (val set): [0.0, 0.0, 0.0, 0.0, 0.0]
Mean IOU (val set): 0.0


## Test


In [ ]:
# nuovo TEST


model_instance.eval()  # imposta il modello in modalità valutazione

all_mious = []

with torch.no_grad():  # disabilita il calcolo dei gradienti per l'inferenza

    for batch_idx, batch in enumerate(loader["test_loader"]):

        if batch_idx == 1: # forbice strana
         continue

        # Il tuo batch può restituire più elementi; qui estraiamo ciò che serve
        data, data1, targets, modelid, modelcat = batch

        # Sposta su GPU (o sul device corretto) e converti in float se necessario
        data = data.float().cuda()
        targets = targets.float().cuda()

        # Inferenza con il modello
        afford_pred = model_instance(data)   # [batch_size, num_points, 2] ad es.

        # Se la tua uscita è di shape [B, N, 2], prendi l'argmax per ricavare la classe
        afford_pred = torch.argmax(afford_pred, dim=-1)  # Shape: [1, 2048]

        afford_pred = afford_pred ^ 1


        new_targets = targets[:, :, 0]

        # Convertilo su CPU per calcolo mIoU (se la tua funzione lo richiede)
        afford_pred_cpu = afford_pred.detach().cpu().numpy()
        ground_truth = new_targets.detach().cpu().numpy()  # [B, N]

        # Calcolo del mIOU (supponendo che la tua funzione handle batch e shape correttamente)
        miou = calculate_miou(afford_pred_cpu, ground_truth, threshold=0.05)
        all_mious.append(miou)

# Al termine, puoi calcolare la media delle metriche
mean_miou = np.mean(all_mious)
print(f" IOU (test set): {all_mious}")
print(f"Mean IOU (test set): {mean_miou}")

 IOU (test set): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Mean IOU (test set): 0.0


# Extension

In [6]:
class RandomApplySubset:
    def __init__(self, transforms, n_select):
        """
        Initialize with a list of transformations and the number of transformations to apply.
        :param transforms: List of possible transformations.
        :param n_select: Number of transformations to randomly select and apply.
        """
        self.transforms = transforms
        self.n_select = n_select

    def __call__(self, image):
        """
        Randomly selects a subset of transformations and applies them to the image.
        :param image: Input image to transform.
        :return: Transformed image.
        """
        selected_transforms = random.sample(self.transforms, self.n_select)
        for transform in selected_transforms:
            #print(f"Applying {transform} to image of type {type(image)}")
            image = transform(image)
        return image


class BackgroundTransform:
    def __init__(self, backgrounds, blur_sigma=(0.1, 2.0)):
        """
        Args:
            backgrounds (list of torch.Tensor): List of background images as tensors.
            blur_sigma (tuple): Min and max sigma for Gaussian blur.
        """
        if not isinstance(backgrounds, list) or len(backgrounds) == 0:
            raise ValueError("BackgroundTransform requires a non-empty list of background tensors.")
        if not all(isinstance(bg, torch.Tensor) for bg in backgrounds):
            raise TypeError("All backgrounds must be torch tensors.")

        self.backgrounds = backgrounds
        self.blur_sigma = blur_sigma

    def __call__(self, rendered_images):
        """
        Args:
            rendered_images (torch.Tensor)[]: Input images tensor of shape (B, C, H, W).

        Returns:
            torch.Tensor: Transformed image blended with a random background.
        """
        transformed_images = []

        for rendered_image in rendered_images:
            # Select a random background
            bg = random.choice(self.backgrounds)
            bg = bg.to(rendered_image.device)

            # Resize the background to match the rendered image
            bg = transforms.Resize(rendered_image.shape[-2:])(bg)

            # Apply Gaussian blur to the background
            blurred_bg = F.gaussian_blur(bg, kernel_size=(5, 5), sigma=random.uniform(*self.blur_sigma))

            # Create a mask from the rendered image
            mask = (rendered_image.sum(dim=0, keepdim=True) > 0).float()  # Mask identifies non-zero pixels

            # Blend the rendered image with the blurred background
            transformed_image = rendered_image * mask + blurred_bg * (1 - mask)

            # Ensure output is valid
            if transformed_image.ndim == 2:
                transformed_image = transformed_image.unsqueeze(0)
            transformed_image = transformed_image.to(rendered_image.device)

            transformed_images.append(transformed_image)

        transformed_images = torch.stack(transformed_images).to(rendered_images.device)

        return transformed_images

In [18]:
# extension su parte 3


model_instance_ext = None

learning_rate = 0.0001
n_layers = 4
n_views = 4
n_augs = 3

prompt= "A 3D rendering of a pair of scissors, emphasizing the ergonomic handles, which are shaped for a firm and comfortable one-handed grip"

clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

backgrounds = [
    transforms.ToTensor()(Image.open("./data/bg1.jpg").resize((224, 224))).to(device),
    transforms.ToTensor()(Image.open("./data/bg2.jpg").resize((224, 224))).to(device),
    transforms.ToTensor()(Image.open("./data/bg3.jpg").resize((224, 224))).to(device),
    torch.tensor((1., 1., 1.)).to(device)
]

possible_augmentations = [
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5), # gia usata
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2)) # new  introduce variazioni di rotazione, traslazione e scala

]

# Custom augmentation that randomly applies a subset
augment_transform = transforms.Compose([
    RandomApplySubset(possible_augmentations, n_select=3),
    clip_normalizer
])


# Ottieni il primo batch dal val_loader
first_batch = next(iter(loader["val_loader"]))

data, data1, targets, modelid, modelcat = first_batch
vertex = data
obj_path = 'alpha.obj'
pointToMesh(data, obj_path)




model_instance_ext = optimize(obj_path, learning_rate, n_layers, n_views, n_augs, prompt, augment_transform=augment_transform)


Background image path:  ./data/desk1.jpg


  0%|          | 1/1000 [00:00<05:46,  2.88it/s]

Last 100 CLIP score: -0.27001953125


 10%|█         | 101/1000 [00:29<04:32,  3.30it/s]

Last 100 CLIP score: -0.288148193359375


...

 80%|████████  | 801/1000 [03:53<00:56,  3.50it/s]

Last 100 CLIP score: -0.281114501953125


 90%|█████████ | 901/1000 [04:22<00:28,  3.51it/s]

Last 100 CLIP score: -0.2847900390625


100%|██████████| 1000/1000 [04:52<00:00,  3.42it/s]


## validation

In [21]:
# nuovo eval


model_instance_ext.eval()  # imposta il modello in modalità valutazione

all_mious = []

with torch.no_grad():  # disabilita il calcolo dei gradienti per l'inferenza

    for batch_idx, batch in enumerate(loader["val_loader"]):

        if batch_idx == 1: #questo perchè il primo valore lo uso per il train ( VA BENE ???????--- TODO ############)
         continue

        # Il tuo batch può restituire più elementi; qui estraiamo ciò che serve
        data, data1, targets, modelid, modelcat = batch

        # Sposta su GPU (o sul device corretto) e converti in float se necessario
        data = data.float().cuda()
        targets = targets.float().cuda()

        # Inferenza con il modello
        afford_pred = model_instance_ext(data)   # [batch_size, num_points, 2] ad es.

        # Se la tua uscita è di shape [B, N, 2], prendi l'argmax per ricavare la classe
        afford_pred = torch.argmax(afford_pred, dim=-1)  # Shape: [1, 2048]

        afford_pred = afford_pred ^ 1


        new_targets = targets[:, :, 0]

        # Convertilo su CPU per calcolo mIoU (se la tua funzione lo richiede)
        afford_pred_cpu = afford_pred.detach().cpu().numpy()
        ground_truth = new_targets.detach().cpu().numpy()  # [B, N]

        # Calcolo del mIOU (supponendo che la tua funzione handle batch e shape correttamente)
        miou = calculate_miou(afford_pred_cpu, ground_truth, threshold=0.05)
        all_mious.append(miou)

# Al termine, puoi calcolare la media delle metriche
mean_miou = np.mean(all_mious)
print(f" IOU (val set): {all_mious}")
print(f"Mean IOU (val set): {mean_miou}")

 IOU (val set): [0.8603712671509282, 0.7659574468085106, 0.4767267267267267, 0.44312267657992566, 0.4876237623762376]
Mean IOU (val set): 0.6067603759284657


## test


In [22]:
# nuovo TEST


model_instance_ext.eval()  # imposta il modello in modalità valutazione

all_mious = []

with torch.no_grad():  # disabilita il calcolo dei gradienti per l'inferenza

    for batch_idx, batch in enumerate(loader["test_loader"]):

        if batch_idx == 1: # forbice strana
         continue

        # Il tuo batch può restituire più elementi; qui estraiamo ciò che serve
        data, data1, targets, modelid, modelcat = batch

        # Sposta su GPU (o sul device corretto) e converti in float se necessario
        data = data.float().cuda()
        targets = targets.float().cuda()

        # Inferenza con il modello
        afford_pred = model_instance_ext(data)   # [batch_size, num_points, 2] ad es.

        # Se la tua uscita è di shape [B, N, 2], prendi l'argmax per ricavare la classe
        afford_pred = torch.argmax(afford_pred, dim=-1)  # Shape: [1, 2048]

        afford_pred = afford_pred ^ 1


        new_targets = targets[:, :, 0]

        # Convertilo su CPU per calcolo mIoU (se la tua funzione lo richiede)
        afford_pred_cpu = afford_pred.detach().cpu().numpy()
        ground_truth = new_targets.detach().cpu().numpy()  # [B, N]

        # Calcolo del mIOU (supponendo che la tua funzione handle batch e shape correttamente)
        miou = calculate_miou(afford_pred_cpu, ground_truth, threshold=0.05)
        all_mious.append(miou)

# Al termine, puoi calcolare la media delle metriche
mean_miou = np.mean(all_mious)
print(f" IOU (test set): {all_mious}")
print(f"Mean IOU (test set): {mean_miou}")

 IOU (test set): [0.9263233190271817, 0.9569413511507052, 0.5978181818181818, 0.6489122280570142, 0.7271329746348962, 0.724429416737109, 0.8079937304075235, 0.7477477477477478, 0.9151785714285714]
Mean IOU (test set): 0.7836086134454366
